In [1]:
import supervision as sv
from ultralytics import YOLO
import cv2
import numpy as np

In [2]:
model = YOLO("yolo11n.pt")

In [3]:
cap = cv2.VideoCapture("4791734-hd_1920_1080_30fps.mp4")
cap.read()
cap.read()
ret, frame = cap.read()
cv2.imwrite("frame.jpg", frame)   # or pass frame directly to OpenCV window
cap.release()

In [4]:
points = []

def click(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x, y))
        print('point added')

cv2.namedWindow("Draw Polygon")
cv2.setMouseCallback("Draw Polygon", click)

while True:
    img = frame.copy()
    if points:
        cv2.polylines(img, [np.array(points)], isClosed=False, color=(0,255,0), thickness=2)
    cv2.imshow("Draw Polygon", img)
    key = cv2.waitKey(10) & 0xFF 
    if key == ord("q"):
        break

cv2.destroyAllWindows()
print("Final polygon coordinates:", points)

point added
point added
point added
point added
point added
point added
point added
point added
Final polygon coordinates: [(123, 674), (326, 675), (29, 720), (328, 718), (636, 877), (1550, 840), (713, 1021), (1736, 945)]


In [5]:
print(points)

[(123, 674), (326, 675), (29, 720), (328, 718), (636, 877), (1550, 840), (713, 1021), (1736, 945)]


In [30]:
polygon_left = sv.PolygonZone(
    polygon= np.array([[123, 674], [326, 675], [328, 718], [29, 720]])
)

polygon_bottom = sv.PolygonZone(
    polygon= np.array([[636, 877], [1550, 840], [1936, 945], [713, 1021]])
)


zone_annotator_left = sv.PolygonZoneAnnotator(
    zone = polygon_left,
    color = sv.Color.RED
)

zone_annotator_bottom = sv.PolygonZoneAnnotator(
    zone = polygon_bottom,
    color = sv.Color.BLUE
)

bounding_box_annotator = sv.BoxAnnotator()

attrs = [a for a in dir(polygon_left) if not a.startswith("_")]
print("polygon_left attrs:\n", attrs)


polygon_left attrs:
 ['current_count', 'frame_resolution_wh', 'mask', 'polygon', 'trigger', 'triggering_anchors']


In [20]:
tracker = sv.ByteTrack()


In [34]:
video_path = "4791734-hd_1920_1080_30fps.mp4"
cap = cv2.VideoCapture(video_path)

In [35]:
left_zone_ids= set()
bottom_zone_ids = set()

In [36]:

while True: 
    ret, frame = cap.read()
    if not ret: 
        break
    
    results = model(frame, verbose=False)[0]
    
    detections = sv.Detections.from_ultralytics(results)
    
    vehicle_classes = [2, 3, 5, 7]
    mask = np.isin(detections.class_id, vehicle_classes)
    detections = detections[mask]    
    tracked = tracker.update_with_detections(detections)        
    
    
    left_mask = polygon_left.trigger(tracked)
    bottom_mask = polygon_bottom.trigger(tracked)
    
    
    for tid, in_left, in_bottom in zip(tracked.tracker_id, left_mask, bottom_mask):
        if in_left:
            left_zone_ids.add(tid)
        if in_bottom:
            bottom_zone_ids.add(tid)
    
    annotated = frame.copy()
    
    annotated = bounding_box_annotator.annotate(scene=annotated, detections=detections)
    
    
    annotated = zone_annotator_bottom.annotate(annotated)
    annotated = zone_annotator_left.annotate(annotated)
    
    cv2.imshow("Tracking", annotated)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
    
print("Left zone cumulative count:", len(left_zone_ids))
print("Bottom zone cumulative count:", len(bottom_zone_ids))

    
cap.release()
cv2.destroyAllWindows()
    

Left zone cumulative count: 2
Bottom zone cumulative count: 19
